[View in Colaboratory](https://colab.research.google.com/github/Santosh-Gupta/Book2Vec/blob/master/GoodReadsDataClean.ipynb)

In [0]:
import re
from collections import Counter
import time

import numpy as np

from __future__ import print_function
import collections
import math
import numpy as np
from numpy import genfromtxt
import random
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

import os

from urllib.request import urlretrieve
from os.path import isfile, isdir


import pandas as pd

!pip install -U -q PyDrive

from google.colab import files
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os
import sys


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
dl_id = input("Enter Gdrive file ID for ratings: ") #1zA50dJB7ZYFPTvk4nHjvZX7jMxXio54w 

# DOWNLOAD ZIP
print ("Downloading  file")
myzip = drive.CreateFile({'id': dl_id})
myzip.GetContentFile('ratings.csv')

Enter Gdrive file ID for ratings: 1zA50dJB7ZYFPTvk4nHjvZX7jMxXio54w


In [0]:
print( os.listdir() )

['datalab', 'DlRefinedRatings.csv', '.cache', 'checkpointsBook2VecCbowWindow2', 'checkpointsBook2VecCbowWindow2Downloaded', '.forever', 'model.zip', 'books.csv', '.keras', '.local', '.ipython', '.config', 'ratings.csv']


In [0]:
%matplotlib inline

In [0]:
r = pd.read_csv( 'ratings.csv' )
r.book_id.max()

In [0]:
reviews_per_user = r.groupby( 'user_id' ).user_id.apply( lambda x: len( x ))
reviews_per_user.describe()

count    53424.000000
mean       111.868804
std         26.071224
min         19.000000
25%         96.000000
50%        111.000000
75%        128.000000
max        200.000000
Name: user_id, dtype: float64

In [0]:
r.sort_values('rating') 

,user_id,book_id,rating
3489851,35383,1257,1
137788,3766,122,1
924756,14673,14,1
924758,14673,65,1
924760,14673,29,1
3216008,8227,7066,1
924766,14673,83,1
3216031,7225,7582,1
4393169,20080,1892,1
137795,3779,131,1


In [0]:
r45 = r[r.rating >= 0]  #only have ratings 4 or higher 
r45.sort_values('rating') 

In [0]:
reviews_per_user45 = r45.groupby( 'user_id' ).user_id.apply( lambda x: len( x ))
reviews_per_user45.describe()  #desribed the number of times a user pops up, so the number of books a user has reviewed. 

count    53424.000000
mean       111.868804
std         26.071224
min         19.000000
25%         96.000000
50%        111.000000
75%        128.000000
max        200.000000
Name: user_id, dtype: float64

In [0]:
r45over5 = r45.groupby('user_id').filter(lambda x: len(x) >= 5) #only have users who appear more than x times. Only include users who have at leasts x ratings
                                                                                                  #So to consider y
                                                                                                 #books to left and right, have x=y*2+1. 7 for cbow window 3, 5 for cbow window 2, etc

In [0]:
reviews_per_user45over5 = r45over5.groupby( 'user_id' ).user_id.apply( lambda x: len( x ))
reviews_per_user45over5.describe()

count    53424.000000
mean       111.868804
std         26.071224
min         19.000000
25%         96.000000
50%        111.000000
75%        128.000000
max        200.000000
Name: user_id, dtype: float64

In [0]:
reviews_per_user45over5BookId = r45over5.groupby( 'book_id' ).user_id.apply( lambda x: len( x ))
reviews_per_user45over5BookId.describe()

count    10000.000000
mean       597.647900
std       1267.289788
min          8.000000
25%        155.000000
50%        248.000000
75%        503.000000
max      22806.000000
Name: user_id, dtype: float64

In [0]:
r45over3Over10B = r45over5.groupby('book_id').filter(lambda x: len(x) >= 15) #only have have that appear more than x times

In [0]:
reviews_per_user45over5BookIdOver10 = r45over3Over10B.groupby( 'book_id' ).user_id.apply( lambda x: len( x ))
reviews_per_user45over5BookIdOver10.describe()

count     9998.000000
mean       597.765553
std       1267.389242
min         24.000000
25%        155.000000
50%        248.000000
75%        503.000000
max      22806.000000
Name: user_id, dtype: float64

In [0]:
r45over3Over10B.to_csv('refinedRatingsForCbowWindow3.csv') #To make it into a regular dataframe object again

In [0]:
rr2 = pd.read_csv( 'refinedRatingsForCbowWindow3.csv' )

rr3= rr2.groupby(['user_id'], as_index=False, sort=True)


print(rr3.apply(lambda x: x.head(20)))



               Unnamed: 0  user_id  book_id  rating
0     0                 0        1      258       5
      75               75        1      268       3
      76               76        1     5556       3
      77               77        1     3638       3
      78               78        1     1796       5
      79               79        1      867       3
      80               80        1       47       3
      81               81        1     2738       3
      230483       230483        1     4691       4
      230484       230484        1      238       2
      230485       230485        1     2063       4
      230486       230486        1      916       3
      230487       230487        1     4614       1
      230488       230488        1      111       3
      230489       230489        1       11       5
      230490       230490        1     1644       5
      230491       230491        1     3889       3
      230492       230492        1      136       5
      230493

In [0]:
print(rr2['user_id'].max())


53424


In [0]:
mydict={n : n for n in random.sample(range(1, rr2['user_id'].max()+1), rr2['user_id'].max())}


In [0]:
randomColumn1 = dict()
i=1
for n in random.sample(range(1, rr2['user_id'].max()+1), rr2['user_id'].max()):
  randomColumn1.update({i:n})
  i=i+1

  
rr2['SortIndex1'] = rr2['user_id'].map(randomColumn1)

#rr2['SortIndex2'] = rr2['user_id'].map(randomColumn2)

rr2['SortIndex2'] = np.random.randint(1, 500, rr2.shape[0])

In [0]:
rr2.head()

,Unnamed: 0,user_id,book_id,rating,SortIndex1,SortIndex2
0,0,1,258,5,11157,99
1,1,2,4081,4,37155,112
2,2,2,260,5,37155,349
3,3,2,9296,5,37155,196
4,4,2,2318,3,37155,13


In [0]:
r3 = rr2.sort_values(['SortIndex1', 'SortIndex2']) 

r3.head(50)

#after this, get the header index 

,Unnamed: 0,user_id,book_id,rating,SortIndex1,SortIndex2
6436,6436,322,1748,3,1,5
232549,232549,322,1282,2,1,6
7222,7222,322,123,3,1,11
6360,6360,322,364,4,1,20
6439,6439,322,5349,4,1,22
6359,6359,322,358,5,1,40
6362,6362,322,157,5,1,47
231989,231989,322,2193,3,1,49
7880,7880,322,650,3,1,51
6465,6465,322,1580,3,1,53


In [0]:
#For more info on what the code below is doing
#https://stackoverflow.com/questions/50892819/pandas-how-to-get-a-row-count-by-the-value-of-a-particular-column-value-and-ad?answertab=oldest#tab-top

r3['Headernumber'] = np.where(r3['user_id'].duplicated(), -1, 
                         r3.groupby('user_id')['user_id'].transform(len))

r3.head(20)

,Unnamed: 0,user_id,book_id,rating,SortIndex1,SortIndex2,Headernumber
6436,6436,322,1748,3,1,5,107
232549,232549,322,1282,2,1,6,-1
7222,7222,322,123,3,1,11,-1
6360,6360,322,364,4,1,20,-1
6439,6439,322,5349,4,1,22,-1
6359,6359,322,358,5,1,40,-1
6362,6362,322,157,5,1,47,-1
231989,231989,322,2193,3,1,49,-1
7880,7880,322,650,3,1,51,-1
6465,6465,322,1580,3,1,53,-1


In [0]:
r3.to_csv('RefinedRatingsCbowW2ForAnalogies2nd.csv', index=False) #may have to removed the index = false

In [0]:
# 2. Create & upload a file text file.
uploadModel = drive.CreateFile()

#zipname = 'RatingRefined'
#zipfolder(zipname, 'checkpoints5')
uploadModel.SetContentFile("RefinedRatingsCbowW2ForAnalogies2nd.csv") #name of file located in local disk 
uploadModel.Upload()